# RAGの基本的な動作
RAG:Retrieval Augmented Generation(検索拡張生成)は外部ナレッジを元に回答を生成するLLMの使い方です。RAGは大きく分けて画像の様に３つのステップから成り立っています。

[画像が表示されない場合はこちら](images/foundation.png)

![](images/foundation.png)


# 事前準備

In [1]:
!python -m pip install openai
import json
from openai import AzureOpenAI

In [52]:
# 次の３つの値をご自身のものに置き換えてください。Replace the following three values with your resources. 
deployment = '<your-aoai-deployment-name>'
apikey = '<your-aoai-api-key>'
base = 'https://<your-aoai-resource-name>.openai.azure.com'
api_version="2023-12-01-preview"

In [53]:
client = AzureOpenAI(
  azure_endpoint = base, 
  api_key=apikey,  
  api_version=api_version
)

# 質問

In [54]:
question="出張で精算できる経費について教えてください。"

# Step1.クエリ化

In [55]:
query_template = """
あなたは、企業内のナレッジベースにアクセスできます。ユーザーからの質問に基づいて検索クエリを生成してください。
- 検索クエリ語には、引用ファイル名や文書名（info.txtやdoc.pdfなど）を含めないでください。
- 検索語のみを表示し、引用符などは出力しないでください。
- 検索クエリ語に[]や<>で囲まれたテキストを含めない。
- 特殊文字を含めない。
- 質問が英語以外の場合、質問で使用されている言語で検索クエリを生成する。
- 検索クエリを生成できない場合は、数字の0のみを返す。

質問：私のヘルスプランについて教えてください。
クエリ：ヘルスプラン

質問：私のヘルスプランには有酸素運動は含まれていますか？
クエリ：ヘルスプラン 有酸素運動 適用範囲
質問：{question}
クエリ：
"""

response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "user", "content": query_template.format(question=question)},
    ],
    temperature=0
)

query = response.choices[0].message.content
print(query)

出張 経費 精算


# Step2.クエリによる検索
何を検索対象とするかはシナリオによって変わります。
例えばインターネット上の情報から検索したい場合はBing APIやGoogle APIを使用します。社内のナレッジベースを検索したい場合はAzure AI SearchやGraph APIを使用します。
このサンプルでは仕組みの理解のために固定リストとしてナレッジを定義してクエリの部分一致で検索をします。

In [56]:
data_source = [
    {"filename": "出張規約.pdf", "content": "出張とは本社または支社から移動先の距離が100km以上のことを指します。"},
    {"filename": "経費報告書.pdf", "content": "経費報告書は、出張や業務に係る支出を正確に記録し、承認を得るための文書です。"},
    {"filename": "交通費規程.pdf", "content": "交通費は、公共交通機関の利用が原則とされ、タクシーを使用する場合は事前に承認を得る必要があります。また公共交通機関は鉄道またはバスを基本としますが、鉄道またはバスを利用した移動時間が4時間を超える場合または海外への出張は、飛行機を利用できます。"},
    {"filename": "宿泊手配.pdf", "content": "出張時の宿泊は、一泊1万5000円までを経費精算可能とします。"},
    {"filename": "慶弔規程.pdf", "content": "上司や同僚が慶弔事に関する連絡を受けた場合、速やかに人事または関係部署に報告し、適切な対応を行います。なお、慶弔費は、会社の方針に基づいて妥当な範囲で精算されます。"},
]

search_terms = query.split()
filtered_records = [record for record in data_source if any(term in record["content"] for term in search_terms)]
filtered_records

[{'filename': '出張規約.pdf', 'content': '出張とは本社または支社から移動先の距離が100km以上のことを指します。'},
 {'filename': '経費報告書.pdf',
  'content': '経費報告書は、出張や業務に係る支出を正確に記録し、承認を得るための文書です。'},
 {'filename': '交通費規程.pdf',
  'content': '交通費は、公共交通機関の利用が原則とされ、タクシーを使用する場合は事前に承認を得る必要があります。また公共交通機関は鉄道またはバスを基本としますが、鉄道またはバスを利用した移動時間が4時間を超える場合または海外への出張は、飛行機を利用できます。'},
 {'filename': '宿泊手配.pdf', 'content': '出張時の宿泊は、一泊1万5000円までを経費精算可能とします。'},
 {'filename': '慶弔規程.pdf',
  'content': '上司や同僚が慶弔事に関する連絡を受けた場合、速やかに人事または関係部署に報告し、適切な対応を行います。なお、慶弔費は、会社の方針に基づいて妥当な範囲で精算されます。'}]

# Step3.検索結果から回答を生成

In [57]:
generate_answer_system_template = """
企業内のナレッジベースに関する質問をサポートします。回答は簡潔に。
- 以下の出典リストに記載されている事実のみを回答してください。以下の情報が十分でない場合は、「わからない」と答えてください。
- 以下の出典を使用しない回答は作成しないでください。ユーザーに明確な質問をすることが助けになる場合は、質問してください。
- 表形式の情報については、htmlテーブルとして返してください。マークダウン形式で返さないでください。質問が英語でない場合は、質問で使用されている言語で回答してください。
- 出典からの回答で使用する各事実には、「必ず」filenameを含めてください。出典を参照するには角括弧を使用します。
- 例えば、「filename：出張規約.pdf」のcontentから生成した回答には末尾に[出張規約.pdf]を付与します。
"""

generate_answer_template = """
質問：{question}
出典：{context}
回答：
"""

response = client.chat.completions.create(
    model=deployment, # model = "deployment_name".
    messages=[
        {"role": "system", "content": generate_answer_system_template},
        {"role": "user", "content": generate_answer_template.format(question=question, context=filtered_records)},
    ],
    temperature=0
)

answer = response.choices[0].message.content
print(answer)

出張で精算できる経費は以下の通りです。

1. 交通費：公共交通機関の利用が原則とされています。タクシーを使用する場合は事前に承認を得る必要があります。公共交通機関は鉄道またはバスを基本としますが、鉄道またはバスを利用した移動時間が4時間を超える場合または海外への出張は、飛行機を利用できます[交通費規程.pdf]。

2. 宿泊費：出張時の宿泊は、一泊1万5000円までを経費精算可能とします[宿泊手配.pdf]。

以上の経費は、経費報告書に正確に記録し、承認を得る必要があります[経費報告書.pdf]。
